<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/oai_client_cost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright (c) Microsoft Corporation. All rights reserved. 

Licensed under the MIT License.

# Usage tracking with AtuoGen
## 1. Use AutoGen's OpenAIWrapper for cost estimation
The `OpenAIWrapper` from `autogen` tracks token counts and costs of your API calls. Use the `create()` method to initiate requests and `print_usage_summary()` to retrieve a detailed usage report, including total cost and token usage for both cached and actual requests.

- `mode=["actual", "total"]` (default): print usage summary for non-caching completions and all completions (including cache).
- `mode='actual'`: only print non-cached usage.
- `mode='total'`: only print all usage (including cache).

Reset your session's usage data with `clear_usage_summary()` when needed.

## 2. Track cost and token count for agents
We also support cost estimation for agents. Use `Agent.print_usage_summary()` to print the cost summary for the agent.
You can retrieve usage summary in a dict using `Agent.get_actual_usage()` and `Agent.get_total_usage()`. Note that `Agent.reset()` will also reset the usage summary.

To gather usage data for a list of agents, we provide an utility function `autogen.agent_utils.gather_usage_summary(agents)` where you pass in a list of agents and gather the usage summary.

## Caution when using Azure OpenAI!
If you are using azure OpenAI, the model returned from completion doesn't have the version information. The returned model is either 'gpt-35-turbo' or 'gpt-4'. From there, we are calculating the cost based on gpt-3.5-0613: ((0.0015, 0.002) per 1k prompt and completion tokens) and gpt-4-0613: (0.03,0.06). This means the cost is wrong if you are using the 1106 version of the models from azure OpenAI.

This will be improved in the future. However, the token count summary is accurate. You can use the token count to calculate the cost yourself.

## Requirements

AutoGen requires `Python>=3.8`:
```bash
pip install "pyautogen"
```

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [2]:
!pip install pyautogen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 14.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.7 kB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 17.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 22.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.9/120.9 kB 13.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 18.1 kB/s eta 0:00:0000:0100:02
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11


In [2]:
!pip uninstall scipy
!pip install scipy

Found existing installation: scipy 1.10.1
Uninstalling scipy-1.10.1:
  Would remove:
    /Users/lg/miniconda3/envs/py311/lib/python3.11/site-packages/scipy
    /Users/lg/miniconda3/envs/py311/lib/python3.11/site-packages/scipy-1.10.1-py3.11.egg-info
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [3]:
import autogen
from autogen import OpenAIWrapper
from autogen import AssistantAgent, UserProxyAgent
from autogen.agent_utils import gather_usage_summary

# config_list = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": ["gpt-3.5-turbo", "gpt-4-1106-preview"],
#     },
# )

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-3.5-turbo-1106"],
    },
)

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well).

The config list looks like the following:
```python
config_list = [
    {
        "model": "gpt-4",
        "api_key": "<your OpenAI API key>",
    },  # OpenAI API endpoint for gpt-4
    {
        "model": "gpt-35-turbo-0613",  # 0613 or newer is needed to use functions
        "base_url": "<your Azure OpenAI API base>", 
        "api_type": "azure", 
        "api_version": "2023-08-01-preview", # 2023-07-01-preview or newer is needed to use functions
        "api_key": "<your Azure OpenAI API key>"
    }
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/website/docs/llm_endpoint_configuration.ipynb) for full code examples of the different methods.

## OpenAIWrapper with cost estimation

In [4]:
client = OpenAIWrapper(config_list=config_list)
messages = [
    {"role": "user", "content": "Can you give me 3 useful tips on learning Python? Keep it simple and short."},
]
response = client.create(messages=messages, model="gpt-3.5-turbo", cache_seed=None)
print(response)
print(response.cost)

ChatCompletion(id='chatcmpl-8o3WMtChih6VZFS0BeaQC1LqhKAEE', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='1. Practice regularly: Consistent practice is key to learning Python effectively. Set aside time each day or week to work on coding exercises or projects to reinforce your understanding of Python concepts.\n\n2. Utilize online resources: Take advantage of the abundance of online tutorials, coding exercises, and forums available for Python learners. Sites like Codecademy, Coursera, or Python.org offer free or affordable resources to help you master Python.\n\n3. Break down problems into smaller tasks: When faced with a coding challenge, break it down into smaller, manageable tasks. Tackle each task step by step, ensuring that you understand and test each component before moving on. Breaking down problems will help you approach complex coding challenges with clarity and efficiency.', role='assistant', function_call=None, tool_calls=None), logpr

## Usage Summary for OpenAIWrapper

When creating a instance of OpenAIWrapper, cost of all completions from the same instance is recorded. You can call `print_usage_summary()` to checkout your usage summary. To clear up, use `clear_usage_summary()`.


In [5]:
client = OpenAIWrapper(config_list=config_list)
messages = [
    {"role": "user", "content": "Can you give me 3 useful tips on learning Python? Keep it simple and short."},
]
client.print_usage_summary()  # print usage summary

No usage summary. Please call "create" first.


In [6]:
# The first creation
# By default, cache_seed is set to 41 and enabled. If you don't want to use cache, set cache_seed to None.
response = client.create(messages=messages, model="gpt-35-turbo-1106", cache_seed=41)
client.print_usage_summary()  # default to ["actual", "total"]
client.print_usage_summary(mode="actual")  # print actual usage summary
client.print_usage_summary(mode="total")  # print total usage summary

----------------------------------------------------------------------------------------------------
Usage summary excluding cached usage: 
Total cost: 0.00035
* Model 'gpt-3.5-turbo-0613': cost: 0.00035, prompt_tokens: 25, completion_tokens: 157, total_tokens: 182

All completions are non-cached: the total cost with cached completions is the same as actual cost.
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Usage summary excluding cached usage: 
Total cost: 0.00035
* Model 'gpt-3.5-turbo-0613': cost: 0.00035, prompt_tokens: 25, completion_tokens: 157, total_tokens: 182
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Usage summary including cached usage: 
Total cost: 0.00035
* Mode

In [7]:
# take out cost
print(client.actual_usage_summary)
print(client.total_usage_summary)

{'total_cost': 0.0003515, 'gpt-3.5-turbo-0613': {'cost': 0.0003515, 'prompt_tokens': 25, 'completion_tokens': 157, 'total_tokens': 182}}
{'total_cost': 0.0003515, 'gpt-3.5-turbo-0613': {'cost': 0.0003515, 'prompt_tokens': 25, 'completion_tokens': 157, 'total_tokens': 182}}


In [8]:
# Since cache is enabled, the same completion will be returned from cache, which will not incur any actual cost.
# So actual cost doesn't change but total cost doubles.
response = client.create(messages=messages, model="gpt-35-turbo-1106", cache_seed=41)
client.print_usage_summary()

----------------------------------------------------------------------------------------------------
Usage summary excluding cached usage: 
Total cost: 0.00035
* Model 'gpt-3.5-turbo-0613': cost: 0.00035, prompt_tokens: 25, completion_tokens: 157, total_tokens: 182

Usage summary including cached usage: 
Total cost: 0.0007
* Model 'gpt-3.5-turbo-0613': cost: 0.0007, prompt_tokens: 50, completion_tokens: 314, total_tokens: 364
----------------------------------------------------------------------------------------------------


In [9]:
# clear usage summary
client.clear_usage_summary()
client.print_usage_summary()

No usage summary. Please call "create" first.


In [10]:
# all completions are returned from cache, so no actual cost incurred.
response = client.create(messages=messages, model="gpt-35-turbo-1106", cache_seed=41)
client.print_usage_summary()

----------------------------------------------------------------------------------------------------
No actual cost incurred (all completions are using cache).

Usage summary including cached usage: 
Total cost: 0.00035
* Model 'gpt-3.5-turbo-0613': cost: 0.00035, prompt_tokens: 25, completion_tokens: 157, total_tokens: 182
----------------------------------------------------------------------------------------------------


## Usage Summary for Agents

- `Agent.print_usage_summary()` will print the cost summary for the agent.
- `Agent.get_actual_usage()` and `Agent.get_total_usage()` will return the usage summary in a dict. When an agent doesn't use LLM, they will return None.
- `Agent.reset()` will reset the usage summary.
- `autogen.agent_utils.gather_usage_summary` will gather the usage summary for a list of agents.

In [11]:
assistant = AssistantAgent(
    "assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 600,
        "cache_seed": None,
        "config_list": config_list,
    },
)

ai_user_proxy = UserProxyAgent(
    name="ai_user",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    code_execution_config=False,
    llm_config={
        "config_list": config_list,
    },
    # In the system message the "user" always refers to the other agent.
    system_message="You ask a user for help. You check the answer from the user and provide feedback.",
)
assistant.reset()

math_problem = "$x^3=125$. What is x?"
ai_user_proxy.initiate_chat(
    assistant,
    message=math_problem,
)

ai_user (to assistant):

$x^3=125$. What is x?

--------------------------------------------------------------------------------
assistant (to ai_user):

To find the value of x, we need to take the cube root of both sides of the equation:

$\sqrt[3]{x^3}=\sqrt[3]{125}$

This simplifies to:

$x=5$

So, x is equal to 5.

--------------------------------------------------------------------------------
ai_user (to assistant):

Great job! Your answer is correct. Taking the cube root of both sides of the equation is the correct method to find the value of x. In this case, the cube root of 125 is indeed 5. Well done!

--------------------------------------------------------------------------------
assistant (to ai_user):

Thank you! I'm happy to hear that the explanation was clear. If you have any more questions, feel free to ask!

--------------------------------------------------------------------------------


In [12]:
ai_user_proxy.print_usage_summary()
print()
assistant.print_usage_summary()

Agent 'ai_user':
----------------------------------------------------------------------------------------------------
Usage summary excluding cached usage: 
Total cost: 0.00024
* Model 'gpt-3.5-turbo-0613': cost: 0.00024, prompt_tokens: 99, completion_tokens: 47, total_tokens: 146

All completions are non-cached: the total cost with cached completions is the same as actual cost.
----------------------------------------------------------------------------------------------------

Agent 'assistant':
----------------------------------------------------------------------------------------------------
Usage summary excluding cached usage: 
Total cost: 0.00041
* Model 'gpt-3.5-turbo-0613': cost: 0.00041, prompt_tokens: 165, completion_tokens: 82, total_tokens: 247

All completions are non-cached: the total cost with cached completions is the same as actual cost.
----------------------------------------------------------------------------------------------------


In [13]:
user_proxy = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=2,
    code_execution_config=False,
    default_auto_reply="That's all. Thank you.",
)
user_proxy.print_usage_summary()

No cost incurred from agent 'user'.


In [14]:
print("Actual usage summary for assistant (excluding completion from cache):", assistant.get_actual_usage())
print("Total usage summary for assistant (including completion from cache):", assistant.get_total_usage())

print("Actual usage summary for ai_user_proxy:", ai_user_proxy.get_actual_usage())
print("Total usage summary for ai_user_proxy:", ai_user_proxy.get_total_usage())

print("Actual usage summary for user_proxy:", user_proxy.get_actual_usage())
print("Total usage summary for user_proxy:", user_proxy.get_total_usage())

Actual usage summary for assistant (excluding completion from cache): {'total_cost': 0.0004115, 'gpt-3.5-turbo-0613': {'cost': 0.0004115, 'prompt_tokens': 165, 'completion_tokens': 82, 'total_tokens': 247}}
Total usage summary for assistant (including completion from cache): {'total_cost': 0.0004115, 'gpt-3.5-turbo-0613': {'cost': 0.0004115, 'prompt_tokens': 165, 'completion_tokens': 82, 'total_tokens': 247}}
Actual usage summary for ai_user_proxy: {'total_cost': 0.00024249999999999999, 'gpt-3.5-turbo-0613': {'cost': 0.00024249999999999999, 'prompt_tokens': 99, 'completion_tokens': 47, 'total_tokens': 146}}
Total usage summary for ai_user_proxy: {'total_cost': 0.00024249999999999999, 'gpt-3.5-turbo-0613': {'cost': 0.00024249999999999999, 'prompt_tokens': 99, 'completion_tokens': 47, 'total_tokens': 146}}
Actual usage summary for user_proxy: None
Total usage summary for user_proxy: None


In [27]:
total_usage_summary, actual_usage_summary = gather_usage_summary([assistant, ai_user_proxy, user_proxy])
total_usage_summary

{'total_cost': 0.0006090000000000001,
 'gpt-35-turbo': {'cost': 0.0006090000000000001,
  'prompt_tokens': 242,
  'completion_tokens': 123,
  'total_tokens': 365}}

In [ ]:
user_proxy.get_total_usage()